In [1]:
# Simple imports
import os
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient

# Load config
load_dotenv('config.env')
base_url = os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321')
print(f"🔗 Connecting to: {base_url}")

# Create client
client = LlamaStackClient(base_url=base_url)
print("✅ Client created")


🔗 Connecting to: http://localhost:8321
✅ Client created


In [2]:
# List all available tools
print("🔍 Available MCP Atlassian tools:")
try:
    tools = client.tools.list(toolgroup_id="mcp::atlassian")
    print(f"Found {len(tools)} tools:")
    
    for i, tool in enumerate(tools, 1):
        tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
        print(f"  {i:2d}. {tool_name}")
        
except Exception as e:
    print(f"❌ Error: {e}")


🔍 Available MCP Atlassian tools:
Found 37 tools:
   1. confluence_add_comment
   2. confluence_add_label
   3. confluence_create_page
   4. confluence_delete_page
   5. confluence_get_comments
   6. confluence_get_labels
   7. confluence_get_page
   8. confluence_get_page_children
   9. confluence_search
  10. confluence_update_page
  11. jira_add_comment
  12. jira_add_worklog
  13. jira_batch_create_issues
  14. jira_batch_get_changelogs
  15. jira_create_issue
  16. jira_create_issue_link
  17. jira_create_sprint
  18. jira_delete_issue
  19. jira_download_attachments
  20. jira_get_agile_boards
  21. jira_get_board_issues
  22. jira_get_issue
  23. jira_get_link_types
  24. jira_get_project_issues
  25. jira_get_project_versions
  26. jira_get_sprint_issues
  27. jira_get_sprints_from_board
  28. jira_get_transitions
  29. jira_get_user_profile
  30. jira_get_worklog
  31. jira_link_to_epic
  32. jira_remove_issue_link
  33. jira_search
  34. jira_search_fields
  35. jira_transitio

In [3]:
# Find Jira tools specifically
print("🎫 Jira tools:")
jira_tools = []
for tool in tools:
    tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
    if 'jira' in str(tool).lower():
        jira_tools.append(tool_name)
        print(f"  - {tool_name}")

print(f"\nFound {len(jira_tools)} Jira tools")


🎫 Jira tools:
  - jira_add_comment
  - jira_add_worklog
  - jira_batch_create_issues
  - jira_batch_get_changelogs
  - jira_create_issue
  - jira_create_issue_link
  - jira_create_sprint
  - jira_delete_issue
  - jira_download_attachments
  - jira_get_agile_boards
  - jira_get_board_issues
  - jira_get_issue
  - jira_get_link_types
  - jira_get_project_issues
  - jira_get_project_versions
  - jira_get_sprint_issues
  - jira_get_sprints_from_board
  - jira_get_transitions
  - jira_get_user_profile
  - jira_get_worklog
  - jira_link_to_epic
  - jira_remove_issue_link
  - jira_search
  - jira_search_fields
  - jira_transition_issue
  - jira_update_issue
  - jira_update_sprint

Found 27 Jira tools


In [4]:
# Test 1: Simple Jira Issue Creation
print("🧪 Test 1: Create Jira Issue")
print("="*40)

# Find jira_create_issue tool
create_tool = None
for tool in tools:
    tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
    if 'jira_create_issue' in str(tool).lower():
        create_tool = tool_name
        break

if create_tool:
    print(f"✅ Found tool: {create_tool}")
    
    try:
        # Simple test parameters - using correct invoke_tool method
        result = client.tool_runtime.invoke_tool(
            tool_name=create_tool,
            kwargs={
                "project_key": "KAN",
                "summary": "Test OOM Error - Simple Test",
                "issue_type": "Task",
                "description": "This is a simple test to verify tool calling works.",
                "additional_fields": {}
            }
        )
        print("✅ Tool call successful!")
        print(f"Result: {result}")
        
    except Exception as e:
        print(f"❌ Tool call failed: {e}")
else:
    print("❌ jira_create_issue tool not found")


🧪 Test 1: Create Jira Issue
✅ Found tool: jira_create_issue
✅ Tool call successful!
Result: ToolInvocationResult(content='{"type":"text","text":"{\\n  \\"message\\": \\"Issue created successfully\\",\\n  \\"issue\\": {\\n    \\"id\\": \\"10004\\",\\n    \\"key\\": \\"KAN-2\\",\\n    \\"summary\\": \\"Test OOM Error - Simple Test\\",\\n    \\"url\\": \\"https://sukantadash.atlassian.net/rest/api/2/issue/10004\\",\\n    \\"description\\": \\"This is a simple test to verify tool calling works.\\",\\n    \\"status\\": {\\n      \\"name\\": \\"To Do\\",\\n      \\"category\\": \\"To Do\\",\\n      \\"color\\": \\"blue-gray\\"\\n    },\\n    \\"issue_type\\": {\\n      \\"name\\": \\"Task\\"\\n    },\\n    \\"priority\\": {\\n      \\"name\\": \\"Medium\\"\\n    },\\n    \\"project\\": {\\n      \\"key\\": \\"KAN\\",\\n      \\"name\\": \\"Ocp support\\",\\n      \\"avatar_url\\": \\"https://sukantadash.atlassian.net/rest/api/2/universal_avatar/view/type/project/avatar/10410\\"\\n    },\\n  

In [5]:
# Test 2: Search for existing issues
print("\n🧪 Test 2: Search Jira Issues")
print("="*40)

# Find jira_search_issues tool
search_tool = None
for tool in tools:
    tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
    if 'jira_search' in str(tool).lower():
        search_tool = tool_name
        break

if search_tool:
    print(f"✅ Found tool: {search_tool}")
    
    try:
        # Simple search - using correct invoke_tool method
        result = client.tool_runtime.invoke_tool(
            tool_name=search_tool,
            kwargs={
                "jql": "project = KAN ORDER BY created DESC",
                "max_results": 5
            }
        )
        print("✅ Search successful!")
        print(f"Result: {result}")
        
    except Exception as e:
        print(f"❌ Search failed: {e}")
else:
    print("❌ jira_search tool not found")



🧪 Test 2: Search Jira Issues
✅ Found tool: jira_search
✅ Search successful!
Result: ToolInvocationResult(content='{"type":"text","text":"Error calling tool \'search\'","annotations":null}', error_code=1, error_message=None, metadata=None)


In [ ]:
# Test 3: One-liner test function
def quick_test(tool_name, kwargs):
    """Quick test any tool with given parameters"""
    print(f"\n⚡ Quick test: {tool_name}")
    try:
        result = client.tool_runtime.invoke_tool(tool_name=tool_name, kwargs=kwargs)
        print(f"✅ Success: {result}")
        return result
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

# Example usage:
# quick_test("jira_create_issue", {"project_key": "KAN", "summary": "Test"})
print("💡 Use quick_test(tool_name, kwargs) to test any tool quickly!")


In [ ]:
# Test 4: Create OOM Error Incident (Simple Version)
print("\n🎫 Test 4: Create OOM Error Incident")
print("="*50)

if create_tool:
    oom_incident = {
        "project_key": "KAN",
        "summary": "Pod failing due to OOM error",
        "issue_type": "Incident", 
        "description": "Pod experiencing Out of Memory errors causing failures.",
        "additional_fields": {
            "priority": {"name": "High"},
            "labels": ["oom-error", "pod-failure", "high-priority"]
        }
    }
    
    result = quick_test(create_tool, oom_incident)
    
    if result:
        print("🎉 OOM Error incident created successfully!")
    else:
        print("❌ Failed to create OOM Error incident")
else:
    print("❌ Create tool not available")
